In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader
import json
from datetime import datetime
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM
from apps.email_manager.models.email import EmailAttachment



/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
convo_reader = ConversationReader(
    start_date=datetime(year=2024, month=3, day=11),
    end_date=datetime(year=2024, month=3, day=12),
)
convo_reader.extract_conversation()


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-03-11 00:00:00) while time zone support is active.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-03-12 00:00:00) while time zone support is active.
  warnings.warn(


In [3]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'Surveyor Appointed for Ticket No:' in a_convo_str:
        (full_convo, attachments, all_files) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        insLLM = InsuranceERPLLM()
        insLLM.store_convo_in_vector_store(f"{full_convo}")
        insLLM.search_convo_vector_store()
        insLLM.query_from_final_results(insLLM.convo_query_results)
        print("------------------------------------")
        insLLM.store_documents_in_vector_store(attachments)
        if insLLM.all_documents:
            insLLM.search_documents_vector_store()
            insLLM.query_from_final_results(insLLM.document_query_results)
        # print(json.dumps(insLLM.queries_results, indent=4))
        insLLM.clean_query_results()
        for attchmnt in attachments:
            print(attchmnt)
        # for a_msg in full_convo:
        #     print(json.dumps(a_msg, indent=4))
        insLLM.record_data(all_files)
        print("")
        print("------------------------------------")
        print("")
        # break


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in

------------------------------------
{
    "net_premium": {
        "value": null,
        "type": "double"
    },
    "issue_date": {
        "value": null,
        "type": "datetime"
    },
    "covernote_number": {
        "value": null,
        "type": "string"
    },
    "policy_number": {
        "value": "=GSAPL/24792/2024",
        "type": "string"
    },
    "claim_number": {
        "value": null,
        "type": "string"
    },
    "customer_name": {
        "value": "=GS Associates (PVT) Limited",
        "type": "string"
    },
    "company_name": {
        "value": "= MILLENIUM ENGINEERING (PVT) LTD",
        "type": "string"
    },
    "risk_type": {
        "value": "=Auto Insurance",
        "type": "string"
    },
    "event_type": {
        "value": "claim_intimated",
        "type": "string"
    },
    "sum_insured": {
        "value": "=3,800,000",
        "type": "double"
    },
    "premium_paid_amount": {
        "value": null,
        "type": "double"
    },
  

In [4]:
print(insLLM.queries_results)

{'net_premium': {'value': None, 'type': 'double'}, 'issue_date': {'value': None, 'type': 'datetime'}, 'covernote_number': {'value': None, 'type': 'string'}, 'policy_number': {'value': 'GSAPL/24792/2024', 'type': 'string'}, 'claim_number': {'value': None, 'type': 'string'}, 'customer_name': {'value': 'GS Associates (PVT) Limited', 'type': 'string'}, 'company_name': {'value': 'MILLENIUM ENGINEERING (PVT) LTD', 'type': 'string'}, 'risk_type': {'value': 'Auto Insurance', 'type': 'string'}, 'event_type': {'value': 'claim_intimated', 'type': 'string'}, 'sum_insured': {'value': 3800000.0, 'type': 'double'}, 'premium_paid_amount': {'value': None, 'type': 'double'}, 'claim_intimation_amount': {'value': 26000.0, 'type': 'double'}, 'claim_paid_amount': {'value': None, 'type': 'double'}, 'premium_paid_date': {'value': None, 'type': 'datetime'}, 'claim_intimation_date': {'value': None, 'type': 'datetime'}, 'claim_paid_date': {'value': None, 'type': 'datetime'}}


In [ ]:
{'net_premium': {'value': None, 'type': 'double'}, 'issue_date': {'value': None, 'type': 'datetime'}, 'covernote_number': {'value': None, 'type': 'string'}, 'policy_number': {'value': None, 'type': 'string'}, 'claim_number': {'value': None, 'type': 'string'}, 'customer_name': {'value': 'Minah Babar', 'type': 'string'}, 'risk_type': {'value': 'Electronic Equipment Insurance', 'type': 'string'}, 'event_type': {'value': 'policy_issued', 'type': 'string'}, 'sum_insured': {'value': None, 'type': 'double'}, 'premium_paid_amount': {'value': None, 'type': 'double'}, 'claim_intimation_amount': {'value': None, 'type': 'double'}, 'claim_paid_amount': {'value': None, 'type': 'double'}, 'premium_paid_date': {'value': None, 'type': 'datetime'}, 'claim_intimation_date': {'value': None, 'type': 'datetime'}, 'claim_paid_date': {'value': None, 'type': 'datetime'}}


In [ ]:
from datetime import datetime
import re


for key, value_dict in insLLM.queries_results.items():
    data_type = value_dict["type"]
    data_value = value_dict["value"]
    # value_dict["value"] = insLLM.raw_results[key]
    # continue
    if data_value is not None and type(data_value) is str and "404 Not Found" not in data_value:
        # Clean the data value
        data_values = data_value.split("=")
        cleaned_value = data_values[-1].strip().replace("'", "")

        # Convert the cleaned data value to the specified type
        if data_type == "double":
            try:
                # Extract numbers from the string using regex
                cleaned_value = re.sub(r'[^\d.]', '', cleaned_value)
                cleaned_value = float(cleaned_value)
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "datetime":
            try:
                # Extract date in the format YYYY-MM-DD using regex
                match = re.search(r'\d{4}-\d{2}-\d{2}', cleaned_value)
                if match:
                    cleaned_value = datetime.strptime(match.group(), "%Y-%m-%d")
                else:
                    cleaned_value = None
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "string":
            cleaned_value = str(cleaned_value)
        
        # Update the value in the dictionary
        value_dict["value"] = cleaned_value

print(insLLM.queries_results)

In [ ]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'Remaining Claim Amount' in a_convo_str:
        (full_convo, attachments, all_files) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        # insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        for attchmnt in attachments:
            print(attchmnt)
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break

In [ ]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)